In [ ]:
import pandas as pd
def plan_from_trajectories(t, keep_only_order=False):
  plan = []
  prev_goal = None

  for goal in t['goal_status']:
    if goal != prev_goal:
      plan.append(goal)
      prev_goal = goal
  # Remove duplicates while preserving order
  final_plan = list(dict.fromkeys(plan))
  if keep_only_order: 
    plan_by_numbers = []
    for stop in final_plan: 
      if stop[:6] == 'moving': 
          plan_by_numbers.append(int(stop[-1]))
    return plan_by_numbers
  return final_plan

plan_mapping = {'[1, 6, 4, 2, 3, 5]': 1, '[3, 5, 4, 1, 6, 2]': 2, '[5, 4, 6, 3, 1, 2]': 3, '[2, 6, 1, 5, 4, 3]': 4, '[6, 5, 2, 4, 1, 3]': 5}

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import Counter  
tdata = pd.DataFrame()
id = 0
for i in range(10):
  temp = pd.read_csv(f'./output_robot3data/out{i}.csv')
  temp = temp[['goal_status']] # 'idle'
  # temp = temp[::8].reset_index(drop=True)
  # if len(Counter(temp['goal_status']).keys()) < 12:
  #   continue
  temp.drop_duplicates(subset=['goal_status'], keep='first', inplace=True)
  if temp.shape[0]<10: 
    continue
  temp['id'] = i
  if temp.iloc[-1].goal_status == '(unknown)':
    temp = temp[:-1]
  temp['plan'] = plan_mapping[str(plan_from_trajectories(temp, keep_only_order=True))]
  tdata = pd.concat([tdata, temp])
  
tdata.reset_index(drop=True, inplace=True)
num_predictions=tdata['id'].unique().__len__()
enc = LabelEncoder()

tdata['goal_status'] = enc.fit_transform(tdata[['goal_status']])

In [ ]:
# def check_seq_length(data):
#   for id in data['id'].unique():
#     print(f'{id}: {len(data[data["id"] == id])}')
#     print(Counter(data[data["id"] == id]['goal_status']))
# check_seq_length(data)

In [23]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import Counter  
data = pd.DataFrame()
id = 0
valid_plans = [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 25, 32, 35, 40, 55]
for i in valid_plans:
  temp = pd.read_csv(f'./output_robot5data/out{i}.csv')
  temp = temp[['goal_status']] # 'idle'
  temp.drop_duplicates(subset=['goal_status'], keep='first', inplace=True)
  if temp.shape[0]<10: 
    continue
  plan = plan_from_trajectories(temp, keep_only_order=True)
  # # print(plan)
  if str(plan) not in list(plan_mapping.keys()):
    continue
  # temp = temp[::8].reset_index(drop=True)
  # if len(Counter(temp['goal_status']).keys()) < 12:
  #   continue
  # if temp.shape[0] < 200: 
  #   continue
  # print(Counter(temp['goal_status']).keys())
  temp['id'] = i
  temp['plan'] = plan_mapping[str(plan)]
  if temp.iloc[-1].goal_status == '(unknown)':
    temp = temp[:-1]

  data = pd.concat([data, temp])
data.reset_index(drop=True, inplace=True)
num_predictions=data['id'].unique().__len__()
enc = LabelEncoder()

data['goal_status'] = enc.fit_transform(data[['goal_status']])

/home/elinasyrr/.conda/envs/pytorch_forecasting/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
data

In [ ]:
# for id in data[data['plan'] == 3]['id'].unique():
#   # plot the data in a scatter plot 
#   temp = data[data['id'] == id].goal_status.values
#   print(temp)  


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Assuming your data is in a pandas DataFrame called 'data'
# # and has columns 'id', 'plan', and 'goal_status'
# plt.figure(figsize=(8, 6))  # Adjust figure size as needed

# plan_id = 1
# all_ids = data[data['plan'] == plan_id]['id'].unique()
# num_ids = len(all_ids)
# offset_scale = 0.1  # Adjust this value to control the offset

# for i, id_val in enumerate(all_ids):
#     goal_statuses = data[data['id'] == id_val]['goal_status']
#     x_values = range(len(goal_statuses))
#     offset = (i - num_ids / 2) * offset_scale  # Create a unique offset for each ID
#     plt.scatter(x_values, goal_statuses + offset, label=f'ID: {id_val}', marker='o', linestyle='-')

# plt.xlabel("Order of Goal Status")
# plt.ylabel("Goal Status Value (with Offset)")
# plt.title(f"Goal Status Progression for Plan: {plan_id}")
# plt.xticks(x_values)
# plt.yticks(sorted(data['goal_status'].unique())) # Show unique goal status values
# plt.legend(title='ID', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.grid(True)
# plt.tight_layout() # Adjust layout to prevent labels from overlapping
# plt.show()

In [ ]:
import numpy as np
horizon = 3
def prepare_data(data):
  col = 'goal_status' #list(data.columns[1:])
  X, y = [], []
  for id in data['id'].unique():
    t = data[data['id'] == id].drop(['id'], axis=1)
    X.append(np.array(t[:-horizon][col]))
    y.append(np.array(t[-horizon:][col]))
  return X,y

In [ ]:
# Assuming data_encoded is your DataFrame
X_train_raw, y_train_raw = prepare_data(data)
X_test_raw, y_test_raw = prepare_data(tdata)
# Find the maximum length in X_train_raw
max_length = max(len(x) for x in X_train_raw)
tmax_length = max(len(x) for x in X_test_raw)

In [ ]:
# Pad the sequences in X_train_raw
import torch 

input = torch.tensor(y_train_raw, dtype=torch.long)
X_train_tensor = [torch.tensor(arr, dtype=torch.long) for arr in X_train_raw]
X_train_tensor = torch.nn.utils.rnn.pad_sequence(X_train_tensor, padding_side='left', batch_first=True).reshape(len(data['id'].unique()), max_length)

tinput = torch.tensor(y_test_raw, dtype=torch.long)
X_test_tensor = [torch.tensor(arr, dtype=torch.long) for arr in X_test_raw]
X_test_tensor = torch.nn.utils.rnn.pad_sequence(X_test_tensor, padding_side='left',batch_first=True).reshape(len(tdata['id'].unique()), tmax_length)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

class EncoderRNN(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super(EncoderRNN, self).__init__()
    self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
    
  def forward(self, input_x):
    o, h = self.gru(input_x)
    return o,h

class DecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(DecoderRNN, self).__init__()
    self.gru = nn.GRU(output_size, hidden_size, batch_first=True)
    self.out = nn.Linear(hidden_size, output_size)

  def forward(self, encoder_hidden, target_tensor): 
    output, hidden = self.gru(target_tensor, encoder_hidden)
    output_logits = self.out(output)
    return output_logits, hidden

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report
def evaluate_model(encoder, decoder, test_loader, device, target_length=400):
  encoder.eval()
  decoder.eval()
  all_predictions = []
  all_labels = []  
  with torch.no_grad():
    for batch_x, batch_y in test_loader:
      batch_size_current = batch_x.size(0)
      encoded_input = torch.nn.functional.one_hot(batch_x, num_classes=16).float().to(device)
      batch_predictions = torch.zeros(batch_size_current, target_length, dtype=torch.long).to(device)
      encoder_outputs, encoder_hidden = encoder(encoded_input)
      decoder_hidden = encoder_hidden
      decoder_input = torch.zeros(batch_size_current, 1, 16).to(device)
      for t in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_hidden, decoder_input)
        predicted_token = torch.argmax(decoder_output[:, -1, :], dim=1)
        batch_predictions[:, t] = predicted_token
        next_input_token = torch.nn.functional.one_hot(predicted_token, num_classes=16).unsqueeze(1).float()
        decoder_input = torch.cat((decoder_input, next_input_token), dim=1)
        decoder_hidden = decoder_hidden
      for m in range(batch_size_current):
        pred_li = list(int(num) for num in batch_predictions[m].cpu().numpy())
        print(f'Prediction: {enc.inverse_transform(pred_li)}')
        true_li = batch_y[m].cpu().numpy()
        print(f'Actual: {enc.inverse_transform(true_li)}\n')
      all_predictions.extend(batch_predictions.cpu().flatten().numpy())
      all_labels.extend(batch_y.flatten().numpy())
    all_predictions = [int(x) for x in all_predictions]
    all_labels = [int(x) for x in all_labels]
    print(all_predictions[:100])
    print(all_labels[:100])
  f1_macro = f1_score(all_labels, all_predictions, average='macro')
  cm = confusion_matrix(all_labels, all_predictions)    
  cr = classification_report(all_labels, all_predictions)
  print(f'Classification Report:\n{cr}')
  print(f'Confusion Matrix:\n{cm}')
  print(f'F1 Macro: {f1_macro}')
  return np.array(all_labels), np.array(all_predictions)


In [ ]:
def evaluate_baseline(test_loader, target_length=100):
    """
    Evaluates a baseline model that extrapolates the last value of the input.
    """
    all_baseline_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_size_current = batch_x.size(0)
            baseline_predictions = torch.zeros(batch_size_current, target_length, dtype=torch.long)
            print(batch_x.shape)
            # Get the last value of the input sequence for each batch
            last_input_values = batch_x[:, -1]  # Assuming batch_x is one-hot encoded
            print(last_input_values)
            # Extrapolate the last value for the target length
            for t in range(target_length):
                baseline_predictions[:, t] = last_input_values
            # for m in range(batch_size_current):
            #   print(f'Prediction: {baseline_predictions[m].cpu().numpy()}')
            #   print(f'Actual: {batch_y[m].cpu().numpy()}')

            all_baseline_predictions.extend(baseline_predictions.cpu().flatten().numpy())
            all_labels.extend(batch_y.flatten().numpy())

    all_baseline_predictions = [int(x) for x in all_baseline_predictions]
    all_labels = [int(x) for x in all_labels]

    f1_macro_baseline = f1_score(all_labels, all_baseline_predictions, average='macro', zero_division=0)
    cm_baseline = confusion_matrix(all_labels, all_baseline_predictions)
    cr_baseline = classification_report(all_labels, all_baseline_predictions, zero_division=0)

    print("\n--- Baseline Evaluation (Last Value Extrapolation) ---")
    print(f'Classification Report:\n{cr_baseline}')
    print(f'Confusion Matrix:\n{cm_baseline}')
    print(f'F1 Macro: {f1_macro_baseline}')

    return np.array(all_labels), np.array(all_baseline_predictions)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
torch.manual_seed(42)
hidden_size = 64
batch_size = 10
classes = 16
n_epochs = 500
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = EncoderRNN(input_size=classes, hidden_size=hidden_size).to(device)
decoder = DecoderRNN(hidden_size=hidden_size, output_size=classes).to(device)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
train_dataset = TensorDataset(X_train_tensor, input)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
loss_per_epoch=[]
for epoch in range(1, n_epochs + 1):
  total_loss = 0 
  for batch_x, batch_y in train_loader:
    padded = torch.cat(
    (
        torch.zeros(batch_size, 1, 16),
        torch.nn.functional.one_hot(batch_y, num_classes=16),
    ),
    dim=1,
    )[:, :-1, :]
    encoded_input=torch.nn.functional.one_hot(batch_x, num_classes=16).float()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    encoder_outputs, encoder_hidden = encoder(encoded_input.to(device))
    decoder_outputs, _ = decoder(encoder_hidden, padded.to(device))
    loss = torch.nn.functional.cross_entropy(decoder_outputs.permute(0,2,1), batch_y.to(device))
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    total_loss += loss.item()
  loss_per_epoch.append(total_loss)
  print(f'Epoch: {epoch}/Loss: {total_loss}')


In [ ]:
test_dataset = TensorDataset(X_test_tensor, tinput)
test_loader =  DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

all_labels, predicted = evaluate_model(encoder, decoder, test_loader, device, target_length=horizon)

In [ ]:
_, _ =evaluate_baseline(test_loader, target_length=horizon)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_per_epoch)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

predicted = predicted.reshape(-1, 2)
all_labels = X_test_tensor
num_sequences = batch_y.shape[0]
time_points = batch_y.shape[1]
predicted_timepoints = predicted.shape[1]

fig, axes = plt.subplots(num_sequences, 1, figsize=(10, num_sequences * 3))  # Adjust figsize as needed

for i in range(num_sequences):
    ax = axes[i]
    # Plot actual batch_y values for the entire sequence
    ax.scatter(range(time_points), all_labels[i], label='Actual (batch_y)')

    # Plot predicted values for the last 2 timepoints
    start_predicted_index = time_points - predicted_timepoints
    ax.scatter(range(start_predicted_index, time_points), predicted[i], color='red', marker='o', label='Predicted')

    ax.set_title(f'Sequence {i+1}')
    ax.set_xlabel('Time Point')
    ax.set_ylabel('Value')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(actual_labels, predicted_labels, alpha=0.5)
plt.xlabel("Actual Y Values")
plt.ylabel("Predicted Y Values")
plt.title("Scatter Plot of Actual vs. Predicted Y Values")
plt.grid(True)
plt.show()